# Лабораторная работа №2  "Алгоритмы классификации"

**ФИО**: *Трофимов М.А.*

**Группа**: *М8О-308Б-18*

**Номер по списку**: *24*

**Вариант**: *24%2+1=1*


# Подготовка данных

Установка необходимых библиотек при необходимости.

In [1]:
%pip install pandas
%pip install numpy
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Загрузка и подкотовка датасета. Загружаем датасет, удаляем се строки содержащие неизвестные данные. Затем удаляем лишние столбцы, которые либо связанные с имеющимися, либо лишены смысла(fnlwgt). Также заменяем названия классов на 1 и 0, для дальнейшей удобной работы.

In [2]:
import pandas as pd
data = pd.read_csv("./adult.csv", na_values = "?")
data = data.dropna()
data.reset_index(inplace=True, drop=True)
data=data.drop(labels=['education', 'capital-loss', 'capital-gain', 'marital-status', 'fnlwgt'], axis=1)

categ_columns = ['workclass', 'occupation', 'relationship', 'race', 'gender', 'native-country']
data[categ_columns] = data[categ_columns].apply(lambda col:pd.Categorical(col).codes)

income = {'<=50K': 1, '>50K': 0}
data['income'] = pd.Categorical(data['income']).rename_categories(income)    
data

,age,workclass,educational-num,occupation,relationship,race,gender,hours-per-week,native-country,income
0,25,2,7,6,3,2,1,40,38,1
1,38,2,9,4,0,4,1,50,38,1
2,28,1,12,10,0,4,1,40,38,0
3,44,2,10,6,0,2,1,40,38,0
4,34,2,6,7,1,4,1,30,38,1
...,...,...,...,...,...,...,...,...,...,...
45217,27,2,12,12,5,4,0,38,38,1
45218,40,2,9,6,0,4,1,40,38,0
45219,58,2,9,0,4,4,0,40,38,1
45220,22,2,9,0,3,4,1,20,38,1


Теперь у нас все значения в таблице выражены числом.
Теперь разобьём выборку на обучающую и тестовую в пропорции 2 к 1, приведём их к *numpy.array* для удобства работы с ними и разобъем отдельно на параметры(X) и метки класса(Y).  

In [3]:
import numpy as np
shuffled = data.sample(frac=1)
parts = np.array_split(shuffled, 3)
data_learn = pd.concat(parts[0:2])
data_test = parts[2]
data_learn.reset_index(inplace=True, drop=True)
data_test.reset_index(inplace=True, drop=True)

np_data = data.to_numpy()
np_data_learn = data_learn.to_numpy()
np_data_test = data_test.to_numpy()

X_l, Y_l = np_data_learn[:, :-1], np_data_learn[:, -1]
X_t, Y_t = np_data_test[:, :-1], np_data_test[:, -1]

# Логистическая Регрессия

Идея логистической регресси в том, что мы хотим подобрать такие параметры $w, b$ для функции $ \alpha(x,w, b) = \left[\frac{1}{1+\exp^{-(w\cdot x + b)}} >= 0.5\right]$ давала наиболее точное предсказание $\hat{y}$ для настоящих значений $y$. 

Иначе данную задачу можно поставить как задачу минимизации логорифмической функции потерь: $$J(w,b) = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i\cdot\log(\hat{y}) - (1-y_i)\cdot\log(1-\hat{y})\right]$$

Данная задача имеет достаточно простой метод решения: градиентный спуск. В нашем случае производные в нашем градиенте будут численно равны:
$$\frac{\partial L}{\partial w_i} = \frac{x_i \bullet (\hat{y} - y)}{N} \text{и} \frac{\partial L}{\partial b} = \frac{(\hat{y} - y)}{N}  $$ 

Обучение происходит не сразу на всей выборке, а небольшими "батчами".

In [4]:
def sigmoid(z):
    return 1.0/(1+np.exp(-z))

def loss(y, y_pred):
    loss = -np.mean(y*np.log(y_pred) - (1-y)*np.log(1-y_pred))
    return loss

def gradient(X, Y, y_pred):
    m = X.shape[0]
    dw = np.dot(X.T, (y_pred - Y))/m
    db = np.sum(y_pred - Y)/m
    return dw, db

def normalize(X):
    m,n = X.shape
    
    for i in range(n):
        X[:, i] = (X[:, i] - X[:, i].mean())/X[:, i].std()
    return X

def train(X, Y, bs, epochs, lr):
    m,n = X.shape
    w = np.zeros((n,1)) 
    b=0
    Y = Y.reshape(m,1)
    X = normalize(X)
    losses = []
    for ep in range(epochs):
        for i in range((m-1)//bs+1):
            xb = X[i*bs:(i+1)*bs]
            yb = Y[i*bs:(i+1)*bs]
            
            y_pred = sigmoid(np.dot(xb, w) +b)
            
            dw,db = gradient(xb, yb, y_pred)
            
            w-=lr*dw
            b-=lr*db
        losses.append(loss(Y, sigmoid(np.dot(X, w)+b))) 
    return w, b, losses

def predict(X, w, b):
    x = normalize(X)
    pred = sigmoid(np.dot(X, w)+b)
    
    pred_classes = [1 if p>=0.5 else 0 for p in pred]
    return np.array(pred_classes)
            

In [5]:
from sklearn.metrics import confusion_matrix

W, B, losses = train(X_l, Y_l, bs=500, epochs=100, lr=0.1)
Y_pred = predict(X_t, W, B)


tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp/(tp + fp)
rec:float = tp/(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7906992171951704
precision: 0.8241827991113932
recall:    0.9171007327624261


In [6]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression(random_state=0, max_iter=100000).fit(X_l, Y_l)
Y_pred = clf.predict(X_t)

tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp/(tp + fp)
rec:float = tp/(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7906992171951704
precision: 0.8241313660161828
recall:    0.917189017392072


Как видим, получившаяся модель почти не уступает в точности библиотечной моделе sklearn.

# SVM

Основная идея **SVM** или Support Vector Machine состоит в том, что мы хотим найти такую гиперплоскость $w\cdot x + b =0$, чтобы она разделяла пространство на две части. По одну сторону от неё был положительный класс, а по другую - отрицательный, а расстояние от неё, до ближайший представителей из каждого калсса было максимальным. Тогда нашу задачу можно сформулировать через минимизацию функции потерь Хинджи с применением к ней L2 регулезации: $$ L(w, b) = \lambda\| w \|^2 + \frac{1}{N} \sum_{i=1}^{N} \max(0, 1-y_i(w\cdot x_i - b)) $$

Или разбивая на два случая $$L_i(w,b) = \left\{ \begin{array}{cc} \lambda\| w \|^2, & y_i(w\cdot x_i - b)\geq 1\\ \lambda\| w \|^2 + 1 - y_i(w\cdot x_i - b), & else \end{array}\right. $$

Оптимизацию так же можно делать через градиентный спуск. Тогда производные будут равны:
$\frac{\partial L_i(w,b)}{\partial w_j} = \left\{ \begin{array}{cc} 2\lambda\cdot w_j, & y_i(w\cdot x_{i} - b)\geq 1\\ 2\lambda \cdot w_j - y_i\cdot x_{ij}, & else \end{array}\right. $ и $\frac{\partial L_i(w,b)}{\partial b} = \left\{ \begin{array}{cc} 0, & y_i(w\cdot x_i - b)\geq 1\\  y_i, & else \end{array}\right. $

In [7]:
def svm_train(X, Y, lr=0.00001, lam=0.001, epochs=100):
    n, m = X.shape
    y_ = np.where(Y<=0, -1, 1)
    w = np.zeros(m).astype('float128')
    b=0
    for _ in range(epochs):
        for i in range(n):
            if y_[i] * (np.dot(X[i], w)-b) >=1 :
                w -= lr*(2*lam*w)
                #b -= lr*y_[i]
            else:
                w -= lr*(2*lam*w - np.dot(X[i], y_[i]))
                b -= lr*y_[i]
    return w, b
    
def svm_predict(X, w, b, rerange:bool=True):
    #X = normalize(X_)
    classes = np.sign(np.dot(X, w)-b)
    return np.where(classes<=0, 0, 1) if rerange else classes


In [8]:
w, b = svm_train(X_l, Y_l)

Y_pred = svm_predict(X_t, w, b)

tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float  = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp /(tp + fp)
rec:float  = tp /(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7543452301976914
precision: 0.7542689434364994
recall:    0.9983225920367264


In [9]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_l, Y_l)
Y_pred = clf.predict(X_t)


tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp/(tp + fp)
rec:float = tp/(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7993233381982221
precision: 0.8290266328471781
recall:    0.9233689414672905


Как мы видим, обе модели неплохо определяют принадлежность к одному из классов.

# Decision Tree

DT или решающие деревья достаточно интересный классификатор, основной идеей которого является лучшее разделение по одному признаку в каждом узле дерева, так образуя разные области, которые принадлежат одному или другому классу. Лучшее разделения определяется с помощью неопределённонсти Джини(Gini impurity). Оно делается достаточно просто, с точки зрения понимания, мы просто перебираем все возможные разделения для текущего узла и выбираем то, для которого прибыль Джини (Gini gain) наибольшая. Так происходит обучение дерева. Дальше для определения принадлежности к одному или к другому классу мы просто спускаемся вниз по дереву как в бинарном дереве поиска, т.е. делая сравнение признака текущего узла с разделяющим его значением. В нашем случае мы идём влево, если наше значение меньше, и вправо иначе.

Gini impurity:
$G(R) = 1 - \sum_{i=1}^N P(x_i \in R) $

Gini gain:
$G_{gain}(L,R)= G(L+R) - \left( \frac{|L|}{|L+R|} \cdot G(L)+ \frac{|R|}{|L+R|} \cdot G(R)\right)$

In [10]:
# Quick value count calculator
from collections import Counter


class Node: 
    """
    Class for creating the nodes for a decision tree 
    """
    def __init__(
        self, 
        X: np.array,
        Y: np.array,
        min_samples_split=None,
        max_depth=None,
        depth=None
    ):
        # Saving the data to the node 
        self.Y = Y 
        self.X = X

        # Saving the hyper parameters
        self.min_samples_split = min_samples_split if min_samples_split else 20
        self.max_depth = max_depth if max_depth else 5

        # Default current depth of node 
        self.depth = depth if depth else 0

        # Extracting all the features
        self.features = range(self.X.shape[1])

        # Calculating the counts of Y in the node 
        self.counts = Counter(Y)

        # Getting the GINI impurity based on the Y distribution
        self.gini_impurity = self.get_GINI()

        # Sorting the counts and saving the final prediction of the node 
        counts_sorted = list(sorted(self.counts.items(), key=lambda item: item[1]))

        # Getting the last item
        yhat = None
        if len(counts_sorted) > 0:
            yhat = counts_sorted[-1][0]

        # Saving to object attribute. This node will predict the class with the most frequent class
        self.yhat = yhat 

        # Saving the number of observations in the node 
        self.n = len(Y)

        # Initiating the left and right nodes as empty nodes
        self.left = None 
        self.right = None 

        # Default values for splits
        self.best_feature = None 
        self.best_value = None 

    @staticmethod
    def GINI_impurity(y1_count: int, y2_count: int) -> float:
        """
        Given the observations of a binary class calculate the GINI impurity
        """
        # Ensuring the correct types
        if y1_count is None:
            y1_count = 0

        if y2_count is None:
            y2_count = 0

        # Getting the total observations
        n = y1_count + y2_count
        
        # If n is 0 then we return the lowest possible gini impurity
        if n == 0:
            return 0.0

        # Getting the probability to see each of the classes
        p1 = y1_count / n
        p2 = y2_count / n
        
        # Calculating GINI 
        gini = 1 - (p1 ** 2 + p2 ** 2)
        
        # Returning the gini impurity
        return gini

    @staticmethod
    def ma(x: np.array, window: int) -> np.array:
        """
        Calculates the moving average of the given list. 
        """
        return np.convolve(x, np.ones(window), 'valid') / window

    def get_GINI(self):
        """
        Function to calculate the GINI impurity of a node 
        """
        # Getting the 0 and 1 counts
        y1_count, y2_count = self.counts.get(0, 0), self.counts.get(1, 0)

        # Getting the GINI impurity
        return self.GINI_impurity(y1_count, y2_count)

    def best_split(self) -> tuple:
        """
        Given the X features and Y targets calculates the best split 
        for a decision tree
        """
        # Creating a dataset for spliting
        df = pd.DataFrame(self.X)
        df['Y'] = self.Y
        
        # Getting the GINI impurity for the base input 
        GINI_base = self.get_GINI()

        # Finding which split yields the best GINI gain 
        max_gain = 0

        # Default best feature and split
        best_feature = None
        best_value = None

        for feature in self.features:
            
            Xdf = df.sort_values(feature)

            # Sorting the values and getting the rolling average
            xmeans = self.ma(Xdf[feature].unique(), 2)

            for value in xmeans:
                # Spliting the dataset 
                left_counts = Counter(Xdf[Xdf[feature]<value]['Y'])
                right_counts = Counter(Xdf[Xdf[feature]>=value]['Y'])

                # Getting the Y distribution from the dicts
                y0_left, y1_left, y0_right, y1_right = left_counts.get(0, 0), left_counts.get(1, 0), right_counts.get(0, 0), right_counts.get(1, 0)

                # Getting the left and right gini impurities
                gini_left = self.GINI_impurity(y0_left, y1_left)
                gini_right = self.GINI_impurity(y0_right, y1_right)

                # Getting the obs count from the left and the right data splits
                n_left = y0_left + y1_left
                n_right = y0_right + y1_right

                # Calculating the weights for each of the nodes
                w_left = n_left / (n_left + n_right)
                w_right = n_right / (n_left + n_right)

                # Calculating the weighted GINI impurity
                wGINI = w_left * gini_left + w_right * gini_right

                # Calculating the GINI gain 
                GINIgain = GINI_base - wGINI

                # Checking if this is the best split so far 
                if GINIgain > max_gain:
                    best_feature = feature
                    best_value = value 

                    # Setting the best gain to the current one 
                    max_gain = GINIgain

        return (best_feature, best_value)

    def grow_tree(self):
        """
        Recursive method to create the decision tree
        """
        # Making a df from the data 
        df = pd.DataFrame(self.X)
        df['Y'] = self.Y

        # If there is GINI to be gained, we split further 
        if (self.depth < self.max_depth) and (self.n >= self.min_samples_split):

            # Getting the best split 
            best_feature, best_value = self.best_split()

            if best_feature is not None:
                # Saving the best split to the current node 
                self.best_feature = best_feature
                self.best_value = best_value

                # Getting the left and right nodes
                left_df, right_df = df[df[best_feature]<=best_value].copy(), df[df[best_feature]>best_value].copy()

                # Creating the left and right nodes
                left = Node(
                    left_df[self.features].to_numpy(), 
                    left_df['Y'].values, 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split
                    )

                self.left = left 
                self.left.grow_tree()

                right = Node(
                    right_df[self.features].to_numpy(), 
                    right_df['Y'].values, 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split
                )

                self.right = right
                self.right.grow_tree()


    def predict(self, X):
        """
        Batch prediction method
        """
        predictions = []

        for x in X:
            values = {}
            for feature in self.features:
                values.update({feature: x[feature]})
        
            predictions.append(self.predict_obs(values))
        
        return predictions
    
    def predict_obs(self, values: dict) -> int:
        """
        Method to predict the class given a set of features
        """
        cur_node = self
        while cur_node.depth < cur_node.max_depth:
            # Traversing the nodes all the way to the bottom
            best_feature = cur_node.best_feature
            best_value = cur_node.best_value

            if cur_node.n < cur_node.min_samples_split:
                break 

            if (values.get(best_feature) < best_value):
                if self.left is not None:
                    cur_node = cur_node.left
            else:
                if self.right is not None:
                    cur_node = cur_node.right
            
        return cur_node.yhat

In [11]:

# Initiating the Node
root = Node(X_l, Y_l, max_depth=4, min_samples_split=1000)

# Getting teh best split
root.grow_tree()


# Predicting 
Xsubset = X_t.copy()
Y_pred = np.array(root.predict(Xsubset))

tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp/(tp + fp)
rec:float = tp/(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7987926230595728
precision: 0.832078795643818
recall:    0.917365586651364


In [12]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

clf.fit(X_l, Y_l)
Y_pred = clf.predict(X_t)

tn, fp, fn, tp = confusion_matrix(Y_t, Y_pred).ravel()

acc:float = (tn + tp)/(tp+tn + fp + fn)
prec:float = tp/(tp + fp)
rec:float = tp/(tp + fn)
print("accuracy:  {0}\nprecision: {1}\nrecall:    {2}".format(acc, prec, rec))

accuracy:  0.7881783202865862
precision: 0.8311889250814333
recall:    0.901121214796504


Как мы видим, получившаяся модель даже несколько лучше, чем готовая из sklearn. Можно предположить, что у нас просто лучше подобраны гиперпараметры(максимальная глубина дерева и минимальный размер выборки, меньше которого мы не делим), из-за чего наше дерево меньше переобучилось, и получился более хороший результат.

# Вывод

В целом, данная работа научила меня работать с python и библиотеками numpy, pandas и sklearn гораздо лучше. Разобрал три алгоритма классификации Logistic Regression, SVM и Decision tree, написал их реализации и сравнил их работу с библиотечными.